<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/proyectoDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Deep Learning
- Diego Cordova 20212
- Marco Jurado 20308
- Cristian Aguirre 20231
- Paola Contreras 20213
- Paola de Leon 20361

In [1]:
! pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/83.6 kB ? eta -:--:--
     ---------------------------------------  81.9/83.6 kB 1.5 MB/s eta 0:00:01
     ---------------------------------------- 83.6/83.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/78.2 kB ? eta -:--:--
     ---------------------------------------- 78.2/78.2 kB ? eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110713 sha256=a23eeec37a85a306056fb5a4beb7222bddcf601d20a4dffc67ddb72b924c4280
  Stored in directory: c:\users\marco\appdata\local\pip\cache\wheels\43\4b\fb\736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!ls -a /root/

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
!mkdir /root/.kaggle/

La sintaxis del comando no es correcta.


In [4]:
!mv /content/kaggle.json /root/.kaggle/


"mv" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [5]:
ls /root/.kaggle/


El formato del par�metro no es correcto: "root".


In [6]:
!chmod 600 /root/.kaggle/kaggle.json

"chmod" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [7]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

Traceback (most recent call last):
  File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\marco\AppData\Roaming\Python\Python310\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Users\marco\AppData\Roaming\Python\Python310\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "C:\Users\marco\AppData\Roaming\Python\Python310\site-packages\kaggle\api\kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\marco\.kaggle. Or use the environment method.


In [8]:
%cd /content
!unzip face-expression-recognition-dataset.zip

[WinError 2] El sistema no puede encontrar el archivo especificado: '/content'
c:\Users\marco\OneDrive\Desktop\mercaditos\proyectoDL


"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Dividir los videos en frames

In [9]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/38.1 MB ? eta -:--:--
                                              0.1/38.1 MB 2.0 MB/s eta 0:00:20
                                              0.2/38.1 MB 2.9 MB/s eta 0:00:14
                                              0.4/38.1 MB 3.0 MB/s eta 0:00:13
                                              0.5/38.1 MB 3.0 MB/s eta 0:00:13
                                              0.7/38.1 MB 3.3 MB/s eta 0:00:12
                                              0.9/38.1 MB 3.3 MB/s eta 0:00:12
     -                                        1.0/38.1 MB 3.3 MB/s eta 0:00:12
     -                                        1.2/38.1 MB 3.3 MB/s eta 0:00:12
     -                                        1.3/38.1 MB 3.3 MB/s eta 0:00:12
     -                                        1.5/38.1 MB 3.3 MB/s eta 0:00:11
     -                                        1.7/38.1 MB 3.


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Podemos evaluar muchos videos al mismo tiempo y en este proceso volver a frames la cantidad de videos que sea necesaria.

In [10]:
import os
import cv2

# Carpeta que contiene los videos
video_folder = 'validation_videos/'

# Carpeta de salida para guardar los frames
output_folder = 'validation_frames/'

# Crea la carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista de archivos en la carpeta de videos
video_files = os.listdir(video_folder)

for video_file in video_files:
    if video_file.endswith('.mp4') or video_file.endswith('.avi'):
        video_path = os.path.join(video_folder, video_file)

        # Abre el video
        cap = cv2.VideoCapture(video_path)

        # Verifica si el video se abrió correctamente
        if not cap.isOpened():
            print(f"No se pudo abrir el video: {video_path}")
            continue

        frame_count = 0
        frame_interval = 5  # Captura un frame cada 5 frames

        # Lee y guarda los frames como imágenes
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                # Guarda el frame como una imagen
                frame_filename = os.path.join(output_folder, f'{os.path.splitext(video_file)[0]}_frame_{frame_count:04d}.png')
                cv2.imwrite(frame_filename, frame)

            frame_count += 1

        # Cierra el video
        cap.release()

        print(f'Se han guardado {frame_count // frame_interval} frames del video {video_file} como imágenes.')

print('Proceso completo.')


Se han guardado 72 frames del video face_recognition_1.mp4 como imágenes.
Se han guardado 41 frames del video face_recognition_2.mp4 como imágenes.
Proceso completo.
